In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
import torch
import argparse
import json
import os

In [2]:
import torch
import torch_geometric
from torch_geometric.data import Data

In [3]:

with open("../paths.json", "r") as f:
        paths = json.load(f)
        hdf = paths['hdf_dir'] 
        graph = paths['graph_dir']

In [6]:
train_diagnoses = pd.read_hdf(f'{hdf}train/diagnoses.h5',key='table')
val_diagnoses = pd.read_hdf(f'{hdf}val/diagnoses.h5',key='table')
test_diagnoses = pd.read_hdf(f'{hdf}test/diagnoses.h5',key='table')
all_diagnoses = pd.concat([train_diagnoses, val_diagnoses, test_diagnoses], axis=0)

print("the size of all diagnoses is: ", all_diagnoses.shape)

the size of all diagnoses is:  (11698, 124)


In [9]:
args = {
    "k": 3,  # 'Number of nearest neighbors for k_closest mode
    "mode": 'k_closest',  # Graph mode: k_closest or threshold
    "freq_adjust": 'store_true',  # Apply frequency adjustment
}

In [10]:
freq_adjustment = all_diagnoses.sum(axis=0) if args["freq_adjust"] else None

In [12]:
def get_device():
    """Get the best device (CUDA or CPU) for computation."""
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def calculate_score_matrix(diagnoses, freq_adjustment=None, debug=False):
    """Calculate the score matrix based on diagnosis data."""
    print('==> Calculating score matrix')
    device = get_device()
    
    # Convert diagnoses to a PyTorch tensor
    diagnoses = torch.tensor(diagnoses.values, dtype=torch.float16, device=device)
    
    if freq_adjustment is not None:
        freq_adjustment = torch.tensor(freq_adjustment.values, dtype=torch.float16, device=device)
        freq_adjustment = 1 / (freq_adjustment + 1e-8)  # Avoid division by zero
        freq_adjustment = freq_adjustment.unsqueeze(0)  # Make it broadcastable
        diagnoses *= freq_adjustment  # Apply frequency adjustment
    
    if debug:
        diagnoses = diagnoses[:1000]  # Limit data size in debug mode

    num_rows = diagnoses.size(0)
    scores = torch.zeros((num_rows, num_rows), dtype=torch.float16, device=device)
    batch_size = 500
    
    print(f'==> Processing in batches (batch size: {batch_size})...')

    # Compute score matrix in batches
    for start in range(0, num_rows, batch_size):
        end = min(start + batch_size, num_rows)
        batch = diagnoses[start:end]
        scores[start:end] = torch.mm(batch, diagnoses.T)

        # Clear cache to reduce memory pressure
        del batch
        torch.cuda.empty_cache()

    # Convert to CPU numpy array
    scores = scores.cpu().numpy()
    
    
    return scores

In [13]:
# Calculate score matrix
scores = calculate_score_matrix(all_diagnoses, freq_adjustment=freq_adjustment)
print(f'Score matrix shape: {scores.shape}')

==> Calculating score matrix
==> Processing in batches (batch size: 500)...
Score matrix shape: (11698, 11698)


In [14]:

def create_graph_pyg(diagnoses, scores, k=3, penalize=True):
    """
    use the score matrix to create a graph in PyG format
    """
    print('==> Step 1: calculate the  Penalty Similarity ')
    diagnoses = torch.tensor(diagnoses.values).float()
    scores = torch.tensor(scores).float()
    scores.fill_diagonal_(0)  # 去掉自连接

    if penalize:
        diags_per_pt = diagnoses.sum(axis=1)
        total_combined_diags = diags_per_pt.view(-1, 1) + diags_per_pt.view(1, -1)
        scores = 5 * scores - total_combined_diags  # 惩罚项

    print('==> Step 2: select the top k edges')
    edge_index = []
    edge_attr = []

    for i in range(scores.shape[0]):
        k_highest = torch.topk(scores[i], k=k).indices
        for j in k_highest:
            edge_index.append([i, j.item()])
            edge_attr.append(scores[i, j].item())  # 边的权重

    edge_index = torch.tensor(edge_index).T  # 转换为 PyG 格式
    edge_attr = torch.tensor(edge_attr).float()

    print(f'==> generated {len(edge_attr)} edges')

    print('==> Step 3: generate the PyG data object')
    data = Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(diagnoses))

    return data


In [15]:
data = create_graph_pyg(all_diagnoses, scores, k=args["k"], penalize=True)

==> Step 1: calculate the  Penalty Similarity 
==> Step 2: select the top k edges
==> generated 35094 edges
==> Step 3: generate the PyG data object


In [16]:
# Save the graph
graph_path = f'{graph}/diagnosis_graph_{args["mode"]}_k{args["k"]}.pt'
torch.save(data, graph_path)

#locad the graph
# loaded_data = torch.load("graph_data.pt")
 

In [15]:

edges_df = pd.DataFrame({"source": data.edge_index[0].cpu().numpy(), 
                         "target": data.edge_index[1].cpu().numpy(), 
                         "weight": data.edge_attr.cpu().numpy()})

# save the graph
edges_df.to_csv(graph+"graph_edges.csv", index=False)

In [ ]:
# edges_df = pd.read_csv(graph+"graph_edges.csv")

In [16]:
edges_df

,source,target,weight
0,0,84,-12.999998
1,0,2889,-13.000000
2,0,2829,-13.000000
3,1,2889,-5.000000
4,1,2829,-5.000000
...,...,...,...
28348,9449,2829,-4.000000
28349,9449,84,-4.000000
28350,9450,2919,-3.000000
28351,9450,84,-3.000000
